# Project 2: Predicting Credit Card Default




Names:Bryan Angelo Pudjanegara, William Nathanael Widjaja

Student ID:20468612, 20451889



In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as smf
from scipy.stats import chi2_contingency

## Introduction

In recent years, the credit card issuers in China faced  credit card debt crisis and the delinquency is expected to peak in the third quarter.  In order to increase market share, card-issuing banks  over-issued cash and credit cards to unqualified applicants. At the same time, most cardholders, irrespective of their repayment ability, overused credit card for consumption and accumulated heavy credit and cash– card debts. The crisis caused the blow to consumer finance confidence and it is a big challenge for both banks and cardholders.

In a well-developed financial system, crisis management is on the downstream and risk prediction is on the upstream. Credit risk here means the probability of a delay in the repayment of the credit granted . The major purpose of risk prediction is to use financial information, such as business financial statement, customer transaction and repayment records, etc., to predict business performance or individual customers’ credit risk and to reduce the damage and uncertainty.

Many statistical methods, including discriminant analysis, logistic regression, Bayes classifier have been used to develop models of risk prediction. More advanced methods, deep autoencoder, variationa autoencoder can even learn new features which have not been explored but are crucial in accurately improving the accuracy of prediction. 


Logistic regression can be considered a special case of linear regression models. However, the binary response variable violates normality assumptions of general regression models. A logistic regression model specifies that an appropriate function of the fitted probability of the event is a linear function of the observed values of the available explanatory variables. The major advantage of this approach is that it can produce a simple probabilistic formula of classification. 


## Grading

- Completeness (85%): complete all requested questions

- Performance-based score (15%): Performance-based score will be given to good results in Questions 3,4 

## Data


   In this project, we will explore the dataset from Taiwan, which contains information on default payments,  credit data, history of payment, and bill statements of credit card clients in Taiwan from April 2005 to September 2005. Totally, there are 30000 customers' records with 23 features and 1 response (default.payment.next.month)

In [2]:
credit=pd.read_csv("UCI_Credit_Card.csv",index_col=0)
credit.shape

(30000, 24)

In [3]:
credit.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000.0,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
2,120000.0,2,2,2,26,-1,2,0,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
3,90000.0,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
4,50000.0,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
5,50000.0,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [4]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 1 to 30000
Data columns (total 24 columns):
LIMIT_BAL                     30000 non-null float64
SEX                           30000 non-null int64
EDUCATION                     30000 non-null int64
MARRIAGE                      30000 non-null int64
AGE                           30000 non-null int64
PAY_0                         30000 non-null int64
PAY_2                         30000 non-null int64
PAY_3                         30000 non-null int64
PAY_4                         30000 non-null int64
PAY_5                         30000 non-null int64
PAY_6                         30000 non-null int64
BILL_AMT1                     30000 non-null float64
BILL_AMT2                     30000 non-null float64
BILL_AMT3                     30000 non-null float64
BILL_AMT4                     30000 non-null float64
BILL_AMT5                     30000 non-null float64
BILL_AMT6                     30000 non-null float64
PAY_AMT1  

There are 23 features, which include all types of data: categorical , discrete and continuous numerical variables.  



- LIMIT_BAL(numerical): Amount of given credit in NT dollars (includes individual and family/supplementary credit
- SEX(categorical): Gender (1=male, 2=female)
- EDUCATION(categorical): (1=graduate school, 2=university, 3=high school, 4=others, 5=unknown, 6=unknown)
- MARRIAGE(categorical): Marital status (1=married, 2=single, 3=others)
- AGE(numerical): Age in years
- PAY_0, Pay2-Pay6(numerical,discrete) : Repayment status in September, Aug,July, June, May, April, 2005 (-1=pay duly, 1=payment delay for one month, 2=payment delay for two months, ... 8=payment delay for eight months, 9=payment delay for nine months and above)

- BILL_AMT1 to BILL_AMT6(numerical): Amount of bill statement in from  September to April,  2005 (NT dollar)

- PAY_AMT1 to PAY_AMT6 (numerical) : Amount of previous payment  in from September to April, 2005 (NT dollar)

- default.payment.next.month: Default payment (1=yes, 0=no)


#  Questions 


## Q1. Data Munging 
### Import response variable 

In [5]:
Data=pd.DataFrame()
Data['Y']=credit["default.payment.next.month"]

In [6]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 1 to 30000
Data columns (total 1 columns):
Y    30000 non-null int64
dtypes: int64(1)
memory usage: 468.8 KB


### Standardize numerical variables .
For example "BILL_AMT4" is continuous numerical, we want to make this column to take values between 0 and 1 which is called "standardization of data".  We can do this by  

In [7]:
Data["PAY_AMT1"]=(credit["PAY_AMT1"]-credit["PAY_AMT1"].min())/(credit["PAY_AMT1"].max()-credit["PAY_AMT1"].min())
Data["PAY_AMT2"]=(credit["PAY_AMT2"]-credit["PAY_AMT2"].min())/(credit["PAY_AMT2"].max()-credit["PAY_AMT2"].min())
Data["PAY_AMT3"]=(credit["PAY_AMT3"]-credit["PAY_AMT3"].min())/(credit["PAY_AMT3"].max()-credit["PAY_AMT3"].min())
Data["PAY_AMT4"]=(credit["PAY_AMT4"]-credit["PAY_AMT4"].min())/(credit["PAY_AMT4"].max()-credit["PAY_AMT4"].min())
Data["PAY_AMT5"]=(credit["PAY_AMT5"]-credit["PAY_AMT5"].min())/(credit["PAY_AMT5"].max()-credit["PAY_AMT5"].min())
Data["PAY_AMT6"]=(credit["PAY_AMT6"]-credit["PAY_AMT6"].min())/(credit["PAY_AMT6"].max()-credit["PAY_AMT6"].min())

In [8]:
Data["BILL_AMT1"]=(credit["BILL_AMT1"]-credit["BILL_AMT1"].min())/(credit["BILL_AMT1"].max()-credit["BILL_AMT1"].min())
Data["BILL_AMT2"]=(credit["BILL_AMT2"]-credit["BILL_AMT2"].min())/(credit["BILL_AMT2"].max()-credit["BILL_AMT2"].min())
Data["BILL_AMT3"]=(credit["BILL_AMT3"]-credit["BILL_AMT3"].min())/(credit["BILL_AMT3"].max()-credit["BILL_AMT3"].min())
Data["BILL_AMT4"]=(credit["BILL_AMT4"]-credit["BILL_AMT4"].min())/(credit["BILL_AMT4"].max()-credit["BILL_AMT4"].min())
Data["BILL_AMT5"]=(credit["BILL_AMT5"]-credit["BILL_AMT5"].min())/(credit["BILL_AMT5"].max()-credit["BILL_AMT5"].min())
Data["BILL_AMT6"]=(credit["BILL_AMT6"]-credit["BILL_AMT6"].min())/(credit["BILL_AMT6"].max()-credit["BILL_AMT6"].min())

In [9]:
Data["LIMIT_BAL"]=(credit["LIMIT_BAL"]-credit["LIMIT_BAL"].min())/(credit["LIMIT_BAL"].max()-credit["LIMIT_BAL"].min())

In [10]:
Data["AGE"]=(credit["AGE"]-credit["AGE"].min())/(credit["AGE"].max()-credit["AGE"].min())

In [11]:
Data["PAY_0"]=(credit["PAY_0"]-credit["PAY_0"].min())/(credit["PAY_0"].max()-credit["PAY_0"].min())
Data["PAY_2"]=(credit["PAY_2"]-credit["PAY_2"].min())/(credit["PAY_2"].max()-credit["PAY_2"].min())
Data["PAY_3"]=(credit["PAY_3"]-credit["PAY_3"].min())/(credit["PAY_3"].max()-credit["PAY_3"].min())
Data["PAY_4"]=(credit["PAY_4"]-credit["PAY_4"].min())/(credit["PAY_4"].max()-credit["PAY_4"].min())
Data["PAY_5"]=(credit["PAY_5"]-credit["PAY_5"].min())/(credit["PAY_5"].max()-credit["PAY_5"].min())
Data["PAY_6"]=(credit["PAY_6"]-credit["PAY_6"].min())/(credit["PAY_6"].max()-credit["PAY_6"].min())

In [12]:
Data.head()

,Y,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,BILL_AMT5,BILL_AMT6,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
ID,,,,,,,,,,,,,,,,,,,,,
1,1,0.000000,0.000409,0.000000,0.000000,0.000000,0.000000,0.149982,0.069164,0.086723,...,0.080648,0.260979,0.010101,0.051724,0.4,0.4,0.1,0.1,0.0,0.0
2,1,0.000000,0.000594,0.001116,0.001610,0.000000,0.003783,0.148892,0.067858,0.087817,...,0.084074,0.263485,0.111111,0.086207,0.1,0.4,0.2,0.2,0.2,0.4
3,0,0.001738,0.000891,0.001116,0.001610,0.002345,0.009458,0.172392,0.079532,0.093789,...,0.095470,0.272928,0.080808,0.224138,0.2,0.2,0.2,0.2,0.2,0.2
4,0,0.002290,0.001199,0.001339,0.001771,0.002506,0.001892,0.188100,0.111995,0.113407,...,0.109363,0.283685,0.040404,0.275862,0.2,0.2,0.2,0.2,0.2,0.2
5,0,0.002290,0.021779,0.011160,0.014493,0.001615,0.001284,0.154144,0.071601,0.106020,...,0.099633,0.275681,0.040404,0.620690,0.1,0.2,0.1,0.2,0.2,0.2


We will run regressions with multiple features, it is better to let all features take values in similar scales, especially if there are many features. 

**Please standardize all  numerical values and save it in DataFrame, Data**

### Make dummy variables for categorical variables. 
We cannot standardize the categorical variables , because the values of categorical data do not have maximum and minum since they cannot be ranked. Instead we wil use dummy variable (Indicator)

For example 


In [13]:
Data["Sex_dummy"]=[1 if sex==1 else 0 for sex in credit["SEX"]]

In [14]:
Data["Education_dummy"]=[1 if education==1 or education==2 else 0 for education in credit["EDUCATION"]]

In [15]:
Data["Marriage_dummy"]=[1 if marriage==1 else 0 for marriage in credit["MARRIAGE"]]

In [16]:
Data.head()

,Y,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,Sex_dummy,Education_dummy,Marriage_dummy
ID,,,,,,,,,,,,,,,,,,,,,
1,1,0.000000,0.000409,0.000000,0.000000,0.000000,0.000000,0.149982,0.069164,0.086723,...,0.051724,0.4,0.4,0.1,0.1,0.0,0.0,0,1,1
2,1,0.000000,0.000594,0.001116,0.001610,0.000000,0.003783,0.148892,0.067858,0.087817,...,0.086207,0.1,0.4,0.2,0.2,0.2,0.4,0,1,0
3,0,0.001738,0.000891,0.001116,0.001610,0.002345,0.009458,0.172392,0.079532,0.093789,...,0.224138,0.2,0.2,0.2,0.2,0.2,0.2,0,1,0
4,0,0.002290,0.001199,0.001339,0.001771,0.002506,0.001892,0.188100,0.111995,0.113407,...,0.275862,0.2,0.2,0.2,0.2,0.2,0.2,0,1,1
5,0,0.002290,0.021779,0.011160,0.014493,0.001615,0.001284,0.154144,0.071601,0.106020,...,0.620690,0.1,0.2,0.1,0.2,0.2,0.2,1,1,1


The number of categories of EDUCATION is too many (with more categories, the model is less robust when sample size is small). 
- **Please build a dummy variable for EDUCATION:  1 for graduate school and university and 0 else.**
- **Please build a dummy variable for MARRIAGE:  1 for "married" and 0 else.**

## Q2. Weighted Sample
Now let use compute totally, how many custermers default and how many did not default. 


In [17]:
print("Out of ", Data.shape[0], " customers, ", Data['Y'].sum(), " cutomers default!")

Out of  30000  customers,  6636  cutomers default!


Hence our sample is unbalanced  with default:default ratio =1:4 (In balanced data,  1 to 0 ratio should be 50:50 roughly.) For unbalanced data, the estimation of logistic regression parameters is difficult. For example ,a trivial model will predict all customers not to default, because the accuracy rate is still high.  An important step before we slice data into train and test, is to over sample defaulted customers or down sample the undefaulted customers so that the ratio between the undefaulted and defaulted is 50:50 roughly. 


First we will get index for undefault customers and sample 6636 cutomers from this index. 

In [18]:
sampled_undefault=pd.DataFrame()
index=Data[Data["Y"]==0].index
sampled_index=np.random.choice(index,6636,replace=False) # try to get different customers without replacement
sampled_undefault=Data.loc[sampled_index,:]
sampled_undefault.head()

,Y,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,Sex_dummy,Education_dummy,Marriage_dummy
ID,,,,,,,,,,,,,,,,,,,,,
12005,0,0.005724,0.002375,0.000013,0.000000,0.000000,0.000000,0.164744,0.089862,0.101908,...,0.120690,0.2,0.2,0.2,0.2,0.2,0.0,0,1,1
3952,0,0.001717,0.001187,0.001116,0.003221,0.004689,0.005675,0.154672,0.076242,0.092972,...,0.120690,0.2,0.2,0.2,0.2,0.2,0.2,1,1,0
19130,0,0.000000,0.001070,0.001939,0.001884,0.002788,0.002327,0.181442,0.096781,0.102543,...,0.396552,0.3,0.4,0.2,0.2,0.2,0.2,0,0,1
25797,0,0.003434,0.001781,0.003348,0.004831,0.007034,0.005675,0.149174,0.069068,0.087992,...,0.293103,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
1372,0,0.001717,0.002916,0.011556,0.017556,0.008206,0.010390,0.159301,0.080839,0.097262,...,0.155172,0.2,0.2,0.2,0.2,0.2,0.2,0,1,0


Then we take default customers. 

In [19]:
default=pd.DataFrame()
index_1=Data[Data['Y']==1].index
default=Data.loc[index_1,:]
default.head()

,Y,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,Sex_dummy,Education_dummy,Marriage_dummy
ID,,,,,,,,,,,,,,,,,,,,,
1,1,0.000000,0.000409,0.000000,0.000000,0.000000,0.000000,0.149982,0.069164,0.086723,...,0.051724,0.4,0.4,0.1,0.1,0.0,0.0,0,1,1
2,1,0.000000,0.000594,0.001116,0.001610,0.000000,0.003783,0.148892,0.067858,0.087817,...,0.086207,0.1,0.4,0.2,0.2,0.2,0.4,0,1,0
14,1,0.003663,0.000000,0.003348,0.004831,0.003517,0.000000,0.204746,0.130156,0.122417,...,0.155172,0.3,0.4,0.4,0.2,0.2,0.4,1,1,0
17,1,0.003663,0.000000,0.001674,0.000000,0.003868,0.000000,0.160125,0.083312,0.095913,...,0.051724,0.2,0.2,0.4,0.4,0.4,0.4,1,1,0
22,1,0.000362,0.000188,0.000000,0.001018,0.000741,0.000000,0.146799,0.066520,0.086518,...,0.310345,0.1,0.1,0.1,0.1,0.1,0.1,0,1,1


Next we merge two types of customer in new data. 

In [20]:
balanced_data=sampled_undefault.append(default) #0000 then 11111 cuz its appending
print(balanced_data.shape,balanced_data['Y'].sum())
balanced_data.head()

(13272, 24) 6636


,Y,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,Sex_dummy,Education_dummy,Marriage_dummy
ID,,,,,,,,,,,,,,,,,,,,,
12005,0,0.005724,0.002375,0.000013,0.000000,0.000000,0.000000,0.164744,0.089862,0.101908,...,0.120690,0.2,0.2,0.2,0.2,0.2,0.0,0,1,1
3952,0,0.001717,0.001187,0.001116,0.003221,0.004689,0.005675,0.154672,0.076242,0.092972,...,0.120690,0.2,0.2,0.2,0.2,0.2,0.2,1,1,0
19130,0,0.000000,0.001070,0.001939,0.001884,0.002788,0.002327,0.181442,0.096781,0.102543,...,0.396552,0.3,0.4,0.2,0.2,0.2,0.2,0,0,1
25797,0,0.003434,0.001781,0.003348,0.004831,0.007034,0.005675,0.149174,0.069068,0.087992,...,0.293103,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
1372,0,0.001717,0.002916,0.011556,0.017556,0.008206,0.010390,0.159301,0.080839,0.097262,...,0.155172,0.2,0.2,0.2,0.2,0.2,0.2,0,1,0


**Please shuffle the down-sampled data and slice it into train and test. **

In [21]:
train_test_ratio = 0.7
train_length = balanced_data.shape[0] * train_test_ratio #(6636,24) take the 6636
train_length = int(train_length)

balanced_data = balanced_data.sample(frac=1).reset_index(drop=True) #need to shuffle, can drop index number, and reset from 0
train = balanced_data.iloc[:train_length, :]
train.head()

,Y,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,Sex_dummy,Education_dummy,Marriage_dummy
0,0,0.000000,0.000866,0.000000,0.001631,0.005308,0.000000,0.171679,0.092491,0.101939,...,0.741379,0.4,0.5,0.4,0.4,0.2,0.2,1,1,1
1,0,0.001603,0.000772,0.000391,0.000805,0.003868,0.000000,0.154049,0.074978,0.091737,...,0.189655,0.2,0.2,0.2,0.2,0.2,0.2,0,1,0
2,1,0.001029,0.000118,0.000000,0.002349,0.000000,0.002364,0.153126,0.066430,0.086448,...,0.293103,0.3,0.4,0.1,0.4,0.4,0.4,0,1,1
3,1,0.001946,0.001069,0.001674,0.000000,0.003517,0.002837,0.176438,0.099108,0.105889,...,0.086207,0.4,0.4,0.4,0.4,0.4,0.2,0,0,1
4,1,0.021605,0.000000,0.000558,0.034726,0.011723,0.318712,0.147227,0.084131,0.086262,...,0.224138,0.1,0.1,0.1,0.1,0.1,0.1,0,1,1


In [22]:
test = balanced_data.iloc[train_length:, :] 
test = test.reset_index(drop=True) #to make index the leftest part from 0 again
test.head()

,Y,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,Sex_dummy,Education_dummy,Marriage_dummy
0,0,0.002781,0.001531,0.002344,0.003221,0.004689,0.003972,0.210486,0.132349,0.122626,...,0.310345,0.2,0.2,0.2,0.2,0.2,0.2,0,1,0
1,1,0.001145,0.023556,0.001339,0.001960,0.002403,0.000946,0.146945,0.067311,0.108127,...,0.086207,0.1,0.4,0.1,0.2,0.2,0.2,1,1,0
2,0,0.000000,0.000432,0.002315,0.000000,0.001372,0.000000,0.146517,0.066219,0.086743,...,0.293103,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
3,0,0.002290,0.000891,0.002232,0.003221,0.004689,0.002837,0.189864,0.113556,0.112163,...,0.413793,0.2,0.2,0.2,0.2,0.2,0.2,0,0,1
4,1,0.005153,0.002919,0.005028,0.006771,0.009718,0.005864,0.326191,0.199414,0.162816,...,0.396552,0.2,0.2,0.2,0.2,0.2,0.2,0,1,0






## Q3.  Feature selection and Logisitic regression

Applying feature selection, build a robust logistic regression model. The performance will be evaluated with AUC score given that there is no  obvious overfitting. 

In [42]:
# Convert categorical data to numerical
# from pandas.plotting import scatter_matrix
# sm=scatter_matrix(train,figsize=(15,15))

In [1]:
plt.subplots(figsize=(30,10))
sn.heatmap( train.corr(), square=True, annot=True, fmt=".1f" )  

NameError: name 'plt' is not defined

In [25]:
# Pearson Coefficient
train.corr().sort_values("Y")

,Y,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,Sex_dummy,Education_dummy,Marriage_dummy
LIMIT_BAL,-0.202266,0.185700,0.192795,0.200484,0.195475,0.215879,0.231664,0.320670,0.311832,0.317523,...,0.148201,-0.277717,-0.316273,-0.306114,-0.285321,-0.265592,-0.253911,-0.020628,0.132990,0.102318
PAY_AMT1,-0.097557,1.000000,0.471696,0.413796,0.360054,0.190284,0.248395,0.152212,0.264058,0.227869,...,0.025372,-0.081660,-0.079899,-0.012271,-0.020294,-0.014581,-0.013483,-0.009048,0.012046,-0.004371
PAY_AMT2,-0.086593,0.471696,1.000000,0.419566,0.369601,0.162915,0.197300,0.128626,0.122790,0.243887,...,0.022836,-0.068487,-0.059955,-0.068926,-0.010710,-0.002787,-0.008168,0.009925,0.018372,-0.009362
PAY_AMT4,-0.079573,0.360054,0.369601,0.355658,1.000000,0.149205,0.210144,0.158236,0.139731,0.135784,...,0.007474,-0.074511,-0.054977,-0.054151,-0.048188,-0.053283,0.010473,-0.003241,0.040912,-0.004256
PAY_AMT6,-0.075361,0.248395,0.197300,0.171255,0.210144,0.197226,1.000000,0.220624,0.218028,0.223573,...,0.022543,-0.058401,-0.040450,-0.036914,-0.031422,-0.021374,-0.024510,0.002922,0.026740,0.004654
PAY_AMT3,-0.071926,0.413796,0.419566,1.000000,0.355658,0.158789,0.171255,0.149470,0.138824,0.108191,...,0.014435,-0.073216,-0.061383,-0.057142,-0.072478,-0.003735,-0.005696,0.014989,0.034562,-0.006031
PAY_AMT5,-0.066137,0.190284,0.162915,0.158789,0.149205,1.000000,0.197226,0.204904,0.189854,0.201936,...,0.005071,-0.035471,-0.024362,-0.022928,-0.026942,-0.027170,-0.040733,-0.000531,0.042739,-0.008322
BILL_AMT1,-0.028905,0.152212,0.128626,0.149470,0.158236,0.204904,0.220624,1.000000,0.956232,0.913855,...,0.053500,0.168310,0.206422,0.173365,0.165027,0.174475,0.171472,0.027757,0.025468,0.034115
BILL_AMT2,-0.021600,0.264058,0.122790,0.138824,0.139731,0.189854,0.218028,0.956232,1.000000,0.946883,...,0.048722,0.175676,0.211941,0.203918,0.190478,0.196195,0.190007,0.023625,0.026813,0.028992
BILL_AMT4,-0.021037,0.228669,0.229183,0.277099,0.140275,0.196802,0.210453,0.875195,0.903831,0.939546,...,0.044705,0.172221,0.208480,0.202905,0.212793,0.239575,0.229829,0.018060,0.039185,0.029114


In [26]:
# import seaborn as sns

# corr_threshold = 0.05
# # predict_col = train.columns
# predict_col = []
# for col in train.columns:
#     if (col == 'Y'):
#         continue
#     pearson_corr = Data['Y'].corr(Data[col])
#     if (abs(pearson_corr) > corr_threshold):
#         predict_col.append(col)
#         print("Plot of %s against Y (correlated)" % col)
#         plt.scatter(Data[col], Data['Y'])
#         plt.show()
#         plt.close()
#     else:
#         sns.distplot(Data[col])
#         plt.close()
#         temp_data = Data[col, 'Y']
#         temp_data['Probability']
#         print("Plot of %s against Y" % col)
#         plt.scatter(Data[col], Data['Y'])
#         plt.plot(Data[col], Data['Y'], color='r')
#         plt.show()
#         plt.close()

# output = 'Y'

# cols = [ f for f in Data.columns if Data.dtypes[ f ] != "object"]
# cols.remove( 'Y' )

# f = pd.melt( Data, id_vars=output, value_vars=cols)
# g = sns.FacetGrid( f, hue=output, col="variable", col_wrap=5, sharex=False, sharey=False )
# g = g.map( sns.distplot, "value", kde=False).add_legend()

In [27]:
train.columns
predict_col = train.columns[train.columns!='Y']
for i in range(6):
    predict_col = predict_col.drop("PAY_AMT" + str(i+1))
    predict_col = predict_col.drop("BILL_AMT" + str(i+1))
print("Columns to use for prediction", predict_col)

Columns to use for prediction Index(['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
       'PAY_6', 'Sex_dummy', 'Education_dummy', 'Marriage_dummy'],
      dtype='object')


In [28]:
import statsmodels.api as smf

credit_model = smf.Logit(train['Y'],train[predict_col]).fit()

Optimization terminated successfully.
         Current function value: 0.627935
         Iterations 5


In [29]:
credit_model.summary(alpha=0.05)

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      Y   No. Observations:                 9290
Model:                          Logit   Df Residuals:                     9279
Method:                           MLE   Df Model:                           10
Date:                Sat, 02 Nov 2019   Pseudo R-squ.:                 0.09408
Time:                        16:41:01   Log-Likelihood:                -5833.5
converged:                       True   LL-Null:                       -6439.3
Covariance Type:            nonrobust   LLR p-value:                4.633e-254
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
LIMIT_BAL          -2.8427      0.185    -15.360      0.000      -3.205      -2.480
AGE                -0.7489      0.145     -5.155      0.000      -1.034      -0.464
PAY_0               4.0900      0.243     16.813      0.000       3.613       4.567
PAY_2               0.5303      0.295      1.800      0.072      -0.047       1.108
PAY_3               0.3510      0.323      1.087      0.277      -0.282       0.984
PAY_4              -0.1775      0.357     -0.497      0.619      -0.878       0.523
PAY_5              -0.1618      0.394     -0.410      0.682      -0.935       0.611
PAY_6              -0.3165      0.322     -0.983      0.325      -0.947       0.314
Sex_dummy           0.0784      0.045      1.736      0.083      -0.010       0.167
Education_dummy    -0.4691      0.049     -9.616      0.000      -0.565      -0.373
Marriage_dummy      0.1476      0.050      2.954      0.003       0.050       0.245
===================================================================================
"""

In [30]:
credit_model.conf_int(alpha=0.05)

,0,1
LIMIT_BAL,-3.205418,-2.479958
AGE,-1.033593,-0.464107
PAY_0,3.613212,4.566821
PAY_2,-0.047143,1.107707
PAY_3,-0.281603,0.983609
PAY_4,-0.877568,0.522629
PAY_5,-0.934712,0.611070
PAY_6,-0.947337,0.314339
Sex_dummy,-0.010091,0.166799
Education_dummy,-0.564658,-0.373454


In [1]:
train['Prob'] = credit_model.predict(train[predict_col])

NameError: name 'credit_model' is not defined

In [32]:
train['Label']=[1 if x>0.5 else 0 for x in train['Prob']]

C:\Users\Bryan Angelo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [33]:
train.head(3)

,Y,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,BILL_AMT1,BILL_AMT2,BILL_AMT3,...,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,Sex_dummy,Education_dummy,Marriage_dummy,Prob,Label
0,0,0.000000,0.000866,0.000000,0.001631,0.005308,0.000000,0.171679,0.092491,0.101939,...,0.5,0.4,0.4,0.2,0.2,1,1,1,0.723464,1
1,0,0.001603,0.000772,0.000391,0.000805,0.003868,0.000000,0.154049,0.074978,0.091737,...,0.2,0.2,0.2,0.2,0.2,0,1,0,0.562674,1
2,1,0.001029,0.000118,0.000000,0.002349,0.000000,0.002364,0.153126,0.066430,0.086448,...,0.4,0.1,0.4,0.4,0.4,0,1,1,0.502622,1


In [34]:
Predict_table_train=pd.crosstab(train['Label'],train['Y'])

In [35]:
Predict_table_train

Y,0,1
Label,,
0,2707,1394
1,1952,3237


In [36]:
train_PPV = Predict_table_train.loc[1,1]/(Predict_table_train.loc[1,1]+Predict_table_train.loc[1,0])
train_NPV = Predict_table_train.loc[0,0]/(Predict_table_train.loc[0,0]+Predict_table_train.loc[0,1])
print(train_PPV,train_NPV)

0.6238196184235884 0.6600829066081444


## Q4 Fine Tuning of Random Forest Models
Please build random forest classification model  using all features prepared in Q1-Q2. Try to find good parameters of model ([reference](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)) . The performance will be evaluated with AUC score given that there is no obvious overfitting. 

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
def confusionTable(predictedY,realY):
    '''
    predictedY: predicted Y with binary value: 0 or 1
    realY: real Y with binary valye :0 or 1
    '''
    confusion_train=pd.crosstab(predictedY, realY)
    PPV=confusion_train.iloc[1,1]/(confusion_train.iloc[1,1]+confusion_train.iloc[1,0])
    NPV=confusion_train.iloc[0,0]/(confusion_train.iloc[0,0]+confusion_train.iloc[0,1])
    accuracy=(confusion_train.iloc[1,1]+confusion_train.iloc[0,0])/(confusion_train.iloc[0,0]+confusion_train.iloc[0,1]+confusion_train.iloc[1,1]+confusion_train.iloc[1,0])
    confusion_train['PV']=[NPV,PPV]
    specificity=confusion_train.iloc[0,0]/(confusion_train.iloc[1,0]+confusion_train.iloc[0,0])
    sensitivity=confusion_train.iloc[1,1]/(confusion_train.iloc[1,1]+confusion_train.iloc[0,1])
    confusion_train.loc['SS']=[specificity, sensitivity,accuracy]
    return confusion_train

In [39]:
prev_train_acc = 0
prev_test_acc = 0

optimal_n_estimators = 1

for i in range(1, 100):
    forest = RandomForestClassifier(max_depth=5,n_estimators=i, random_state=10).fit(train[predict_col], train['Y'])
    predictedOutcome_train = forest.predict(train[predict_col])
    predictedProb_train = forest.predict_proba(train[predict_col])
    predictedOutcome_test = forest.predict(test[predict_col])
    predictedProb_test = forest.predict_proba(test[predict_col])
    
    ct_train = confusionTable(predictedOutcome_train, train['Y'])
    accuracy_train = ct_train.iloc[2,2]
    ct_test = confusionTable(predictedOutcome_test, test['Y'])
    accuracy_test = ct_test.iloc[2,2]
    
    if (abs(prev_train_acc - accuracy_train) < 0.000000000000001 or abs(prev_test_acc - accuracy_test) < 0.000000000000001):
        print("Optimal number of estimators is %d" % (i - 1))
        optimal_n_estimators = i - 1
        break
    else:
        prev_train_acc = accuracy_train
        prev_test_acc = accuracy_test
    
    print("No. of estimators: %d, Train Accuracy: %.4f, Test Accuracy: %.4f" % (i, accuracy_train, accuracy_test))

No. of estimators: 1, Train Accuracy: 0.7072, Test Accuracy: 0.6929
No. of estimators: 2, Train Accuracy: 0.7096, Test Accuracy: 0.6979
Optimal number of estimators is 2


In [40]:
prev_train_acc = 0
prev_test_acc = 0

optimal_max_features = 1

for i in range(1, predict_col.shape[0] + 1):
    forest = RandomForestClassifier(max_depth=5,max_features=i,n_estimators=optimal_n_estimators, random_state=10).fit(train[predict_col], train['Y'])
    predictedOutcome_train = forest.predict(train[predict_col])
    predictedProb_train = forest.predict_proba(train[predict_col])
    predictedOutcome_test = forest.predict(test[predict_col])
    predictedProb_test = forest.predict_proba(test[predict_col])
    
    ct_train = confusionTable(predictedOutcome_train, train['Y'])
    accuracy_train = ct_train.iloc[2,2]
    ct_test = confusionTable(predictedOutcome_test, test['Y'])
    accuracy_test = ct_test.iloc[2,2]
    
    if (abs(prev_train_acc - accuracy_train) < 0.000000000000001 or abs(prev_test_acc - accuracy_test) < 0.000000000000001):
        print("Optimal max features is %d" % (i - 1))
        optimal_max_features = i - 1
        break
    else:
        prev_train_acc = accuracy_train
        prev_test_acc = accuracy_test
    
    print("No. of estimators: %d, Train Accuracy: %.4f, Test Accuracy: %.4f" % (i, accuracy_train, accuracy_test))

No. of estimators: 1, Train Accuracy: 0.7022, Test Accuracy: 0.6899
No. of estimators: 2, Train Accuracy: 0.7016, Test Accuracy: 0.6934
No. of estimators: 3, Train Accuracy: 0.7096, Test Accuracy: 0.6979
No. of estimators: 4, Train Accuracy: 0.7117, Test Accuracy: 0.6951
No. of estimators: 5, Train Accuracy: 0.7085, Test Accuracy: 0.6974
No. of estimators: 6, Train Accuracy: 0.7096, Test Accuracy: 0.6921
No. of estimators: 7, Train Accuracy: 0.7132, Test Accuracy: 0.6994
No. of estimators: 8, Train Accuracy: 0.7129, Test Accuracy: 0.6959
No. of estimators: 9, Train Accuracy: 0.7124, Test Accuracy: 0.6931
No. of estimators: 10, Train Accuracy: 0.7099, Test Accuracy: 0.6984
No. of estimators: 11, Train Accuracy: 0.7128, Test Accuracy: 0.6883


In [41]:
confusionTable(predictedOutcome_test, test['Y'])

Y,0,1,PV
row_0,,,
0,1585.00000,849.000000,0.651191
1,392.00000,1156.000000,0.746770
SS,0.80172,0.576559,0.688348
